# Preparing ARIA Sentinel-1 data for validation of Solid Earth requirements

**Original code authored by:** David Bekaert, Heresh Fattahi, Eric Fielding, and Zhang Yunjun 

Extensive modifications by Adrian Borsa and Amy Whetter 2022

Reorganized and modified by Ekaterina Tymofyeyeva, March 2024

<div class="alert alert-warning">
Both the initial setup (<b>Prep A</b> section) and download of the data (<b>Prep B</b> section) should be run at the start of the notebook. Methods for validation of transient, secular, and coseismic requirements using Sentinel-1 ARIA data can be run subsequently.
</div>

<hr/>

## Define CalVal Site 

In [ ]:
# Choose a site and track direction
site='MojaveD173' 

# Choose the requirement you are going to validate
requirement='Coseismic' # 'Secular' 'Coseismic' 'Transient'

# What dataset are you processing?
dataset = 'ARIA_S1' # For Sentinel-1 testing with aria-tools

# The date and version of this Cal/Val run
today = '20240528'
version = '1'

# Define your directory structure - you won't need to change this line
start_directory = '/scratch/nisar-st-calval-solidearth' 

# The file where you keep your customized list of sites.
custom_sites = '/home/jovyan/my_sites.txt'

# Please enter a name or username that will determine where your outputs are stored
import os
if os.path.exists('/home/jovyan/me.txt'): # if OpenTopo API key already installed
    with open('/home/jovyan/me.txt') as m:
        you = m.readline().strip()
    print('You are', you)
    print('Using this as the name of the directory where your outputs will be stored.')
    print('Directory structure: start_directory / dataset/ requirement / site / you / today / version ')
else:
    print('We need a name or username (determines where your outputs will be stored)')
    print('Directory structure: start_directory / dataset/ requirement / site / you / today / version ')
    you = input('Please type your name:')
    with open ('/home/jovyan/me.txt', 'w') as m: 
        m.write(you)
    

## Table of Contents:
<a id='prep_TOC'></a>

<hr/>

[**Prep A. Environment Setup**](#prep_a)

[**Prep B. Data Staging**](#prep_b)

[**1. Generate Interferogram Stack**](#prep_gen_ifg)
- [1.1.  Crop Interferograms](#prep_crop_ifg)

[**2. Generation of Time Series from Interferograms**](#prep_gen_ts)
- [2.1. Set Up MintPy Configuration file](#prep_setup_config)
- [2.2. Load Data into MintPy](#prep_load_data)
- [2.3. Validate/Modify Interferogram Network](#secular_validate_network)

<hr/>

<a id='secular_prep_a'></a>
## Prep A. Environment Setup
Setup your environment for processing data

In [ ]:
#Load Packages
import glob
import os
import subprocess
from datetime import datetime as dt
from pathlib import Path
import json
import netrc

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mintpy.cli import view, plot_network
from mintpy.objects import gnss, timeseries
from mintpy.smallbaselineApp import TimeSeriesAnalysis
from mintpy.utils import ptime, readfile, utils as ut
from scipy import signal

from solid_utils.sampling import load_geo, samp_pair, profile_samples, haversine_distance

#Set Global Plot Parameters
plt.rcParams.update({'font.size': 12})

################# Set Directories ##########################################
work_dir = os.path.join(start_directory,dataset,requirement,site,you,today,'v'+version)

print("Work directory:", work_dir)
os.makedirs(work_dir,exist_ok=True)

# Change to Workdir
os.chdir(work_dir)

gunw_dir = os.path.join(work_dir,'products')
os.makedirs(gunw_dir,exist_ok=True)
print("   GUNW    dir:", gunw_dir) 

mintpy_dir = os.path.join(work_dir,'MintPy')
os.makedirs(mintpy_dir,exist_ok=True)
print("   MintPy  dir:", mintpy_dir)
### Change to MintPy workdir
os.chdir(mintpy_dir)

vel_file = os.path.join(mintpy_dir, 'velocity.h5')
msk_file = os.path.join(mintpy_dir, 'maskConnComp.h5')  # maskTempCoh.h5 maskConnComp.h5

with open('/home/jovyan/my_sites.txt','r') as fid:
    sitedata = json.load(fid)
    
print(work_dir)

sitedata['sites'][site]

<a id='secular_prep_b'></a>
## Prep B. Data Staging

In this initial processing step, all the necessary Level-2 unwrapped interferogram products are gathered, organized and reduced to a common grid for analysis with MintPy. Ascending and descending stacks of nearest-neighbor and skip-1 interferograms will be prepared for independent analysis. 

In [ ]:
##################### 1. Download (Aria) Interferograms from ASF ################
os.chdir(work_dir)

# Search for Earthdata login 
fnetrc = '/home/jovyan/.netrc'
earthdata = False
if os.path.exists(fnetrc):
    !chmod 0600 /home/jovyan/.netrc
    #netrc = netrc.netrc()
    remoteHostName  = "urs.earthdata.nasa.gov"
    with open(fnetrc) as file:
        if remoteHostName in file.read():
            authTokens = netrc.authenticators(remoteHostName)
            earthdata_user = authTokens[0]
            earthdata_password = authTokens[2]
            earthdata = True
            
if not earthdata:             
    print('NEEDED To Download ARIA GUNWs: \n Link to create account : https://urs.earthdata.nasa.gov/')
    earthdata_user = input('Please type your Earthdata username:')
    earthdata_password = input('Please type your Earthdata password:')
    with open(fnetrc, 'a') as file:
        file.write('machine urs.earthdata.nasa.gov\n')
        file.write('login ' + earthdata_user + '\n')
        file.write('password ' + earthdata_password)
    !chmod 0600 /home/jovyan/.netrc

print('NEEDED To Download DEMs: \n Link to create account : https://portal.opentopography.org/login')
if os.path.exists('/home/jovyan/.topoapi'): # if OpenTopo API key already installed
    print('OpenTopo API key appears to be installed, using that')
else:
    print('API key location: My Account > myOpenTopo Authorizations and API Key > Request API key')
    opentopography_api_key = input('Please type your OpenTopo API key:')

######################## USE ARIA-TOOLS TO DOWNLOAD GUNW ########################
'''
REFERENCE: https://github.com/aria-tools/ARIA-tools
'''
aria_download = '''ariaDownload.py --num_threads 16 -b {bbox} -u {user} -p {password} -s {start}  -e {end} -t {track} --version 3_0_1 -o Count'''

###############################################################################
print('CalVal site {}'.format(site))
print('  Searching for available GUNW products:\n')

command = aria_download.format(bbox = sitedata['sites'][site]['download_region'],
                               start = sitedata['sites'][site]['download_start_date'],
                               end = sitedata['sites'][site]['download_end_date'],
                               track = sitedata['sites'][site]['sentinel_track'],
                               user = earthdata_user,
                               password = earthdata_password)

process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text = True, shell = True)
print(process.stdout)

############## Download GUNW ##################
print("Start downloading GUNW files ...")
#process = subprocess.run(command.split(' -o')[0], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, shell=True)
os.system(command.split(' -o')[0])
# Missing progressbar
print("Downloaded {} GUNW files in: {}\n".format(len([(x) for x in os.listdir(gunw_dir) if x.endswith('.nc')]), gunw_dir))

############## DO a little CLEANING ###########
data_to_clean = ["avg_rates.csv", "ASFDataDload0.py", "AvgDlSpeed.png", "error.log"]

for i, file in enumerate(data_to_clean):

    if os.path.exists(os.path.join(gunw_dir,file)):
        print('Cleaning unnecessary data {} in {}'.format(file, gunw_dir))
        os.unlink(os.path.join(gunw_dir,file))

#Delete error log file from workdir
print('Cleaning unnecessary data error.log in {}'.format(work_dir))

if os.path.exists(os.path.join(gunw_dir,'error.log')):
    os.unlink(os.path.join(work_dir,"error.log"))

<a id='secular_gen_ifg'></a>
# 1. Generate Interferogram Stack

InSAR time series (i.e., the unfiltered displacement of each pixel vs. time) are estimated from a processed InSAR stack from Section 3.1 (either ascending or descending) using a variant of the small baseline subset (SBAS) approach and then parameterized using the approach described in Section 4. This step uses tools available in the MintPy software package (REF), which provides both SBAS time series and model-based time series parameterization. Recent results on InSAR closure phase and “fading signal” recommend the of use all available interferograms to avoid systematic bias (_Ansari et al._, 2020; _Zheng Y.J. et al._, 2022). As we expect high-quality orbital control for NISAR, we anticipate that the interferogram stack will typically include all nearest-neighbor (i.e., ~12-day pairs) and skip-1 interferograms, which will be the minimum inputs into the SBAS generation step.

We use the open-source ARIA-tools package to download processed L2 interferograms over selected cal/val regions from the Alaska Satellite Facility archive and to stitch/crop the frame-based NISAR GUNW products to stacks that can be directly ingested into MintPy for time-series processing. ARIA-tools uses a phase-minimization approach in the product overlap region to stitch the unwrapped and ionospheric phase, a mosaicing approach for coherence and amplitude, and extracts the geometric information from the 3D data cubes through a mosaicking of the 3D datacubes and subsequent intersection with a DEM. ARIA has been used to pre-process NISAR beta products derived from Sentinel-1 which have revealed interseismic deformation and creep along the San Andreas Fault system, along with subsidence, landsliding, and other signals. 

We use MintPy to validate and modify the InSAR stack, removing interferograms that do not meet minimum coherence criteria, generating a quality control mask for the purpose of identifying noisy pixels within the stack, and referencing estimated deformation to a common location in all interferograms.

<a id='secular_crop_ifg'></a>
## 1.1. Crop Interferograms

In [ ]:
# Crop Interferograms to Analysis Region
os.chdir(work_dir)
mask_file = 'auto'
product_glob = '"'+os.path.join(work_dir,'products','*.nc')+'"'

###########################################################################################################
# Set up ARIA product and mask data with GSHHS water mask:
'''
REQUIRED: Acquire API key to access/download DEMs


Follow instructions listed here to generate and access API key through OpenTopography:
https://opentopography.org/blog/introducing-api-keys-access-opentopography-global-datasets.
'''

if not os.path.exists(os.path.join(work_dir,'stack')):
    if not os.path.exists('/home/jovyan/.topoapi'): # if OpenTopo API key not already installed
        os.system('echo "{api_key}" > /home/jovyan/.topoapi; chmod 600 /home/jovyan/.topoapi'.format(api_key = str(opentopography_api_key)))
    print('Preparing GUNWs for MintPY....')
    if sitedata['sites'][site]['maskWater'] != 'False':
        mask_file = '../mask/watermask.msk'
        command = 'ariaTSsetup.py -f ' + product_glob + ' -b ' + sitedata['sites'][site]['analysis_region'] + ' --mask Download  --croptounion --verbose' # slow
    else: # skip slow mask download when we don't need to mask water
        command = 'ariaTSsetup.py -f ' + product_glob + ' -b ' + sitedata['sites'][site]['analysis_region'] + ' --croptounion --verbose'

    ################################## CROP & PREPARE STACK ###################################################
    print(command)
    os.system(command)
    #result = subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT, text=True, shell=True)
print('Finish preparing GUNWs for MintPy!!')

<a id='secular_gen_ts'></a>
# 2. Creating the MintPy data cube

InSAR time series (i.e., the unfiltered displacement of each pixel vs. time) are estimated from a processed InSAR stack from Section 3.1 (either ascending or descending) using a variant of the small baseline subset (SBAS) approach and then parameterized using the approach described in Section 4. This step uses tools available in the MintPy software package (REF), which provides both SBAS time series and model-based time series parameterization. Recent results on InSAR closure phase and “fading signal” recommend the of use all available interferograms to avoid systematic bias (_Ansari et al._, 2020; _Zheng Y.J. et al._, 2022). As we expect high-quality orbital control for NISAR, we anticipate that the interferogram stack will typically include all nearest-neighbor (i.e., ~12-day pairs) and skip-1 interferograms, which will be the minimum inputs into the SBAS generation step.

We use the open-source ARIA-tools package to download processed L2 interferograms over selected cal/val regions from the Alaska Satellite Facility archive and to stitch/crop the frame-based NISAR GUNW products to stacks that can be directly ingested into MintPy for time-series processing. ARIA-tools uses a phase-minimization approach in the product overlap region to stitch the unwrapped and ionospheric phase, a mosaicing approach for coherence and amplitude, and extracts the geometric information from the 3D data cubes through a mosaicking of the 3D datacubes and subsequent intersection with a DEM. ARIA has been used to pre-process NISAR beta products derived from Sentinel-1 which have revealed interseismic deformation and creep along the San Andreas Fault system, along with subsidence, landsliding, and other signals. 

We use MintPy to validate and modify the InSAR stack, removing interferograms that do not meet minimum coherence criteria, generating a quality control mask for the purpose of identifying noisy pixels within the stack, and referencing estimated deformation to a common location in all interferograms.

<a id='secular_setup_config'></a>
## 2.1. Set Up MintPy Configuration file


The default processing parameters for MintPy's **smallbaselineApp.py** need to be modified by including the following lines in config_file (which must be manually created and placed into mint_dir):

- mintpy.load.processor      = aria
- mintpy.load.unwFile        = ../stack/unwrapStack.vrt
- mintpy.load.corFile        = ../stack/cohStack.vrt
- mintpy.load.connCompFile   = ../stack/connCompStack.vrt
- mintpy.load.demFile        = ../DEM/SRTM_3arcsec.dem
- mintpy.load.incAngleFile   = ../incidenceAngle/{download_start_date}_{download_edn_date}.vrt
- mintpy.load.azAngleFile    = ../azimuthAngle/{download_start_date}_{download_edn_date}.vrt
- mintpy.load.waterMaskFile  = ../mask/watermask.msk
- mintpy.reference.lalo      = auto, or somewhere in your bounding box
- mintpy.topographicResidual.pixelwiseGeometry = no
- mintpy.troposphericDelay.method              = no
- mintpy.topographicResidual                   = no

In [ ]:
####################################################################
### Change to MintPy workdir
os.chdir(mintpy_dir)
config_file = Path(mintpy_dir)/(sitedata['sites'][site]['calval_location'] + '.cfg')

####################################################################
### Write smallbaseline.py config file
config_file_content = """
mintpy.load.processor = aria
mintpy.compute.numWorker = auto
mintpy.load.unwFile = {wd}/stack/unwrapStack.vrt
mintpy.load.corFile = {wd}/stack/cohStack.vrt
mintpy.load.connCompFile = {wd}/stack/connCompStack.vrt
mintpy.load.demFile = {wd}/DEM/glo_90.dem
mintpy.load.incAngleFile = {wd}/incidenceAngle/*.vrt
mintpy.load.azAngleFile = {wd}/azimuthAngle/*.vrt
mintpy.load.waterMaskFile = {mask_file}
mintpy.topographicResidual.pixelwiseGeometry = no
mintpy.troposphericDelay.method = no
mintpy.topographicResidual = no
mintpy.network.tempBaseMax = {tempmax}
mintpy.network.startDate = {startdatenet}
mintpy.network.endDate = {enddatenet}
mintpy.velocity.startDate = {startdatevel}
mintpy.velocity.endDate = {enddatevel}
mintpy.reference.lalo = {reference_lalo}
mintpy.network.excludeIfgIndex = {excludeIfg}""".format(wd = work_dir,
                                                        mask_file = mask_file,
                                                        tempmax=sitedata['sites'][site]['tempBaseMax'],
                                                        excludeIfg=sitedata['sites'][site]['ifgExcludeList'],
                                                        startdatenet=sitedata['sites'][site]['download_start_date'],
                                                        enddatenet=sitedata['sites'][site]['download_end_date'],
                                                        startdatevel=sitedata['sites'][site]['download_start_date'],
                                                        enddatevel=sitedata['sites'][site]['download_end_date'],
                                                        reference_lalo=sitedata['sites'][site]['reference_lalo'])

config_file.write_text(config_file_content)

print('MintPy config file:\n    {}:'.format(config_file))
print(config_file.read_text())

<a id='secular_load_data'></a>
## 2.2. Load Data into MintPy

The output of this step is an "inputs" directory in 'calval_directory/calval_location/MintPy/" containing two HDF5 files:
- ifgramStack.h5: This file contains 6 dataset cubes (e.g. unwrapped phase, coherence, connected components etc.) and multiple metadata
- geometryGeo.h5: This file contains geometrical datasets (e.g., incidence/azimuth angle, masks, etc.)

In [ ]:
#os.chdir(work_dir)
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep load_data'
process = subprocess.run(command, shell=True)
print('Mintpy input files:')
[x for x in os.listdir('inputs') if x.endswith('.h5')]

## 2.3. Clean up! 

Remove downloaded files if desired

In [ ]:
print('Now that you have successfully created the MintPy data cube, you may want to clean up the downloaded products')
cleanup = input('Please type "Yes" if you want to delete the files in the "products" directory:')
if cleanup == "Yes" or cleanup == "YES" or cleanup == "yes":
    import shutil
    shutil.rmtree(gunw_dir)
elif cleanup == "No" or cleanup == "NO" or cleanup == "no":
    print('Keeping your downloaded files')
else: 
    print('ERROR: Please try again. Type "Yes" or "No"')
    